In [14]:
import numpy as np
import threading
import queue
import pandas as pd
#testArray = np.array([[1,2,3,4],[2,3,4,1],[3,4,1,2],[4,1,2,3],[3,4,1,2],[4,1,2,3]])
testArray = np.array([[1, 2], [2, 1]])
np.shape(testArray)

(2, 2)

In [7]:
def simpleNMF(A, k, i):
    m, n = np.shape(A)
    W = np.random.rand(m, k)
    H = np.random.rand(k, n)
    for _ in range(i):
        W = W * (A @ H.T) / (W @ H @ H.T)
        H = H * (W.T @ A) / (W.T @ W @ H)
    return W @ H

simpleNMF(testArray, 2, 1000)

array([[1., 2.],
       [2., 1.]])

In [8]:
def thread_function_W(A, W, H, q, i):
    q.put((i, W * (A @ H.T) / (W @ H @ H.T)))

def thread_function_H(A, W, H, q, i):
    q.put((i, H * (W.T @ A) / (W.T @ W @ H)))

def naiveParallelNMF(A, k, p, numIter):
    m, n = np.shape(A)
    if m % p > 0:
        raise TypeError('Input first dimension not divisible by number of threads')
    if n % p > 0:
        raise TypeError('Input second dimension not divisible by number of threads')
    W = np.random.rand(m, k)
    H = np.random.rand(k, n)
    As1 = [A[int(i*m/p):int((i+1)*m/p), :] for i in range(p)]
    As2 = [A[:, int(i*n/p):int((i+1)*n/p)] for i in range(p)]
    
    for _ in range(numIter):
        Ws = [W[int(i*m/p):int((i+1)*m/p), :] for i in range(p)] # chop up W
        w_threads = []
        w_queue = queue.Queue()
        for j in range(p):
            newThread = threading.Thread(target = thread_function_W, args = (As1[j], Ws[j], H, w_queue, j)) # each thread updates one section
            newThread.start()
            w_threads.append(newThread)
        for thread in w_threads: # wait for all threads to complete
            thread.join()
        while not w_queue.empty(): # reconstitute and update W
            i, v = w_queue.get()
            Ws[i] = v
        W = np.concatenate(Ws, 0)

        Hs = [H[:, int(i*n/p):int((i+1)*n/p)] for i in range(p)] # same procedure for H
        h_threads = []
        h_queue = queue.Queue()
        for j in range(p):
            newThread = threading.Thread(target = thread_function_H, args = (As2[j], W, Hs[j], h_queue, j))
            newThread.start()
            h_threads.append(newThread)
        for thread in h_threads:
            thread.join()
        while not h_queue.empty():
            i, v = h_queue.get()
            Hs[i] = v
        H = np.concatenate(Hs, 1)
    return W @ H
    
naiveParallelNMF(testArray, 2, 2, 10000)

array([[1., 2.],
       [2., 1.]])

In [9]:
def HPC_NMF(A, k, p1, p2, numIter):
    m, n = np.shape(A)
    if m % (p1*p2) > 0:
        raise TypeError('Input first dimension not divisible by number of threads')
    if n % (p1*p2) > 0:
        raise TypeError('Input second dimension not divisible by number of threads')
    W = np.random.rand(m, k)
    H = np.random.rand(k, n)

    Hs = [[np.random.rand(k, int(n/(p1*p2))) for i in range(p1)] for j in range(p2)]


    return A

Next, we will optimize both the number of processor dimensions (p) and the rank of the factorization (k). For each, we will optimize runtime, minimize computing costs, bandwith cost(?), and communication cost (?). We will examine these results on both dense and sparse imput matricies (A)(?). For each set, the study fixes the number of processors at 600 and vary the rank of k. We will replicate this, but also try to optimize the number of processors for our dataset.

Note** we need to find datasets; preferably one sparse and one dense dataset, or we could generate synthetic sparse and dense datasets.



In [10]:
c4 = pd.read_csv('c4_game_database.csv')
c4[c4 < 0] = 2 #changing -1 values to 2
c4
#really not sure if this works for a sparse dataset but wanted to start exploring

pos_01  pos_02  pos_03  pos_04  pos_05  pos_06  pos_07  pos_08  \
0          1.0     1.0     1.0     2.0     2.0     1.0     0.0     2.0   
1          0.0     0.0     1.0     1.0     1.0     1.0     0.0     0.0   
2          0.0     1.0     0.0     1.0     0.0     0.0     0.0     0.0   
3          0.0     0.0     0.0     1.0     0.0     0.0     0.0     0.0   
4          0.0     2.0     2.0     2.0     1.0     0.0     0.0     1.0   
...        ...     ...     ...     ...     ...     ...     ...     ...   
376635     0.0     0.0     0.0     2.0     0.0     0.0     0.0     0.0   
376636     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
376637     0.0     0.0     1.0     0.0     0.0     1.0     0.0     0.0   
376638     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
376639     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   

        pos_09  pos_10  ...  pos_34  pos_35  pos_36  pos_37  pos_38  pos_39  \
0          2.0     2.0  ...     1.0     1.0     2.0     1.0     2.0     1.0   
1          0.0     1.0  ...     2.0     1.0     1.0     2.0     2.0     2.0   
2          2.0     0.0  ...     0.0     2.0     1.0     2.0     2.0     1.0   
3          0.0     0.0  ...     0.0     1.0     1.0     1.0     2.0     1.0   
4          1.0     1.0  ...     0.0     1.0     2.0     2.0     1.0     2.0   
...        ...     ...  ...     ...     ...     ...     ...     ...     ...   
376635     0.0     2.0  ...     0.0     1.0     2.0     2.0     2.0     1.0   
376636     0.0     2.0  ...     0.0     0.0     0.0     2.0     1.0     2.0   
376637     0.0     2.0  ...     1.0     1.0     2.0     2.0     2.0     1.0   
376638     2.0     0.0  ...     2.0     0.0     2.0     2.0     1.0     1.0   
376639     0.0     0.0  ...     2.0     0.0     2.0     2.0     2.0     1.0   

        pos_40  pos_41  pos_42  winner  
0          2.0     1.0     2.0     2.0  
1          1.0     1.0     2.0     1.0  
2          2.0     2.0     2.0     2.0  
3          2.0     0.0     1.0     2.0  
4          1.0     0.0     2.0     1.0  
...        ...     ...     ...     ...  
376635     2.0     0.0     1.0     2.0  
376636     1.0     0.0     0.0     2.0  
376637     1.0     1.0     2.0     2.0  
376638     1.0     2.0     0.0     2.0  
376639     2.0     1.0     2.0     1.0  

[376640 rows x 43 columns]

In [25]:
class NMF_Base(object):

    """ Base class for NMF algorithms
    Specific algorithms need to be implemented by deriving from this class.
    """
    default_max_iter = 100
    default_max_time = np.inf

    def __init__(self):
        raise NotImplementedError(
            'NMF_Base is a base class that cannot be instantiated')

    def set_default(self, default_max_iter, default_max_time):
        self.default_max_iter = default_max_iter
        self.default_max_time = default_max_time

    def run(self, A, k, init=None, max_iter=None, max_time=None, verbose=0):
        """ Run a NMF algorithm
        Parameters
        ----------
        A : numpy.array or scipy.sparse matrix, shape (m,n)
        k : int - target lower rank
        Optional Parameters
        -------------------
        init : (W_init, H_init) where
                    W_init is numpy.array of shape (m,k) and
                    H_init is numpy.array of shape (n,k).
                    If provided, these values are used as initial values for NMF iterations.
        max_iter : int - maximum number of iterations.
                    If not provided, default maximum for each algorithm is used.
        max_time : int - maximum amount of time in seconds.
                    If not provided, default maximum for each algorithm is used.
        verbose : int - 0 (default) - No debugging information is collected, but
                                    input and output information is printed on screen.
                        -1 - No debugging information is collected, and
                                    nothing is printed on screen.
                        1 (debugging/experimental purpose) - History of computation is
                                        returned. See 'rec' variable.
                        2 (debugging/experimental purpose) - History of computation is
                                        additionally printed on screen.
        Returns
        -------
        (W, H, rec)
        W : Obtained factor matrix, shape (m,k)
        H : Obtained coefficient matrix, shape (n,k)
        rec : dict - (debugging/experimental purpose) Auxiliary information about the execution
        """
        info = {'k': k,
                'alg': str(self.__class__),
                'A_dim_1': A.shape[0],
                'A_dim_2': A.shape[1],
                'A_type': str(A.__class__),
                'max_iter': max_iter if max_iter is not None else self.default_max_iter,
                'verbose': verbose,
                'max_time': max_time if max_time is not None else self.default_max_time}
        if init != None:
            W = init[0].copy()
            H = init[1].copy()
            info['init'] = 'user_provided'
        else:
            W = random.rand(A.shape[0], k)
            H = random.rand(A.shape[1], k)
            info['init'] = 'uniform_random'

        if verbose >= 0:
            print ('[NMF] Running: ')
            print (json.dumps(info, indent=4, sort_keys=True))

        norm_A = mu.norm_fro(A)
        total_time = 0

        if verbose >= 1:
            his = {'iter': [], 'elapsed': [], 'rel_error': []}

        start = time.time()
        # algorithm-specific initilization
        (W, H) = self.initializer(W, H)

        for i in range(1, info['max_iter'] + 1):
            start_iter = time.time()
            # algorithm-specific iteration solver
            (W, H) = self.iter_solver(A, W, H, k, i)
            elapsed = time.time() - start_iter

            if verbose >= 1:
                rel_error = mu.norm_fro_err(A, W, H, norm_A) / norm_A
                his['iter'].append(i)
                his['elapsed'].append(elapsed)
                his['rel_error'].append(rel_error)
                if verbose >= 2:
                    print ('iter:' + str(i) + ', elapsed:' + str(elapsed) + ', rel_error:' + str(rel_error))

            total_time += elapsed
            if total_time > info['max_time']:
                break

        W, H, weights = mu.normalize_column_pair(W, H)

        final = {}
        final['norm_A'] = norm_A
        final['rel_error'] = mu.norm_fro_err(A, W, H, norm_A) / norm_A
        final['iterations'] = i
        final['elapsed'] = time.time() - start

        rec = {'info': info, 'final': final}
        if verbose >= 1:
            rec['his'] = his

        if verbose >= 0:
            print ('[NMF] Completed: ')
            print (json.dumps(final, indent=4, sort_keys=True))
        return (W, H, rec)

    def run_repeat(self, A, k, num_trial, max_iter=None, max_time=None, verbose=0):
        """ Run an NMF algorithm several times with random initial values 
            and return the best result in terms of the Frobenius norm of
            the approximation error matrix
        Parameters
        ----------
        A : numpy.array or scipy.sparse matrix, shape (m,n)
        k : int - target lower rank
        num_trial : int number of trials
        Optional Parameters
        -------------------
        max_iter : int - maximum number of iterations for each trial.
                    If not provided, default maximum for each algorithm is used.
        max_time : int - maximum amount of time in seconds for each trial.
                    If not provided, default maximum for each algorithm is used.
        verbose : int - 0 (default) - No debugging information is collected, but
                                    input and output information is printed on screen.
                        -1 - No debugging information is collected, and
                                    nothing is printed on screen.
                        1 (debugging/experimental purpose) - History of computation is
                                        returned. See 'rec' variable.
                        2 (debugging/experimental purpose) - History of computation is
                                        additionally printed on screen.
        Returns
        -------
        (W, H, rec)
        W : Obtained factor matrix, shape (m,k)
        H : Obtained coefficient matrix, shape (n,k)
        rec : dict - (debugging/experimental purpose) Auxiliary information about the execution
        """
        for t in iter(range(num_trial)):
            if verbose >= 0:
                print ('[NMF] Running the {0}/{1}-th trial ...'.format(t + 1, num_trial))
            this = self.run(A, k, verbose=(-1 if verbose is 0 else verbose))
            if t == 0:
                best = this
            else:
                if this[2]['final']['rel_error'] < best[2]['final']['rel_error']:
                    best = this
        if verbose >= 0:
            print ('[NMF] Best result is as follows.')
            print (json.dumps(best[2]['final'], indent=4, sort_keys=True))
        return best

    def iter_solver(self, A, W, H, k, it):
        raise NotImplementedError

    def initializer(self, W, H):
        return (W, H)

<>:147: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:147: SyntaxWarning: "is" with a literal. Did you mean "=="?
/var/folders/xf/8808rrps3wb3r170fzcq_6t00000gn/T/ipykernel_57360/2333205815.py:147: SyntaxWarning: "is" with a literal. Did you mean "=="?
  this = self.run(A, k, verbose=(-1 if verbose is 0 else verbose))


In [26]:
import scipy.sparse as sps
import scipy.optimize as opt
import numpy.linalg as nla
from nonnegfac import matrix_utils as mu
import time
import json
from numpy import random
from nonnegfac.nnls import nnlsm_activeset
from nonnegfac.nnls import nnlsm_blockpivot
class NMF_HALS(NMF_Base):

    """ NMF algorithm: Hierarchical alternating least squares
    A. Cichocki and A.-H. Phan, Fast local algorithms for large scale nonnegative matrix and tensor factorizations,
    IEICE Transactions on Fundamentals of Electronics, Communications and Computer Sciences,
    vol. E92-A, no. 3, pp. 708-721, 2009.
    """

    def __init__(self, default_max_iter=100, default_max_time=np.inf):
        self.eps = 1e-16
        self.set_default(default_max_iter, default_max_time)

    def initializer(self, W, H):
        W, H, weights = mu.normalize_column_pair(W, H)
        return W, H

    def iter_solver(self, A, W, H, k, it):
        AtW = A.T.dot(W)
        WtW = W.T.dot(W)
        for kk in iter(range(0, k)):
            temp_vec = H[:, kk] + AtW[:, kk] - H.dot(WtW[:, kk])
            H[:, kk] = np.maximum(temp_vec, self.eps)

        AH = A.dot(H)
        HtH = H.T.dot(H)
        for kk in iter(range(0, k)):
            temp_vec = W[:, kk] * HtH[kk, kk] + AH[:, kk] - W.dot(HtH[:, kk])
            W[:, kk] = np.maximum(temp_vec, self.eps)
            ss = nla.norm(W[:, kk])
            if ss > 0:
                W[:, kk] = W[:, kk] / ss

        return (W, H)